In [2]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os 
import re


In [10]:
load_dotenv()
slug_collection = 'boredape-baseclub'
api_key = os.getenv('api_key')

url_stats = f'https://api.opensea.io/api/v2/collections/{slug_collection}/stats'
url_nft_overview = f"https://api.opensea.io/api/v2/collections/{slug_collection}"
headers = {
    "accept" : "application/json",
    "x-api-key": api_key
}

<h5> Request stats

In [6]:
try:
    response_stats = requests.get(url_stats, headers=headers).json()
    #response.raise_for_status()
    print(response_stats)
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
    

{'total': {'volume': 60.53499194139634, 'sales': 16534, 'average_price': 0.003661243010850148, 'num_owners': 593, 'market_cap': 45.513550007999996, 'floor_price': 0.005949999998899, 'floor_price_symbol': 'ETH'}, 'intervals': [{'interval': 'one_day', 'volume': 0.0, 'volume_diff': 0.0, 'volume_change': 0.0, 'sales': 0, 'sales_diff': 0.0, 'average_price': 0.0}, {'interval': 'seven_day', 'volume': 0.04554999, 'volume_diff': -20.0, 'volume_change': -0.7900517103441357, 'sales': 10, 'sales_diff': 0.0, 'average_price': 0.004554999}, {'interval': 'thirty_day', 'volume': 0.7483459979886802, 'volume_diff': -120.0, 'volume_change': -0.6078102158771842, 'sales': 115, 'sales_diff': 0.0, 'average_price': 0.006507356504249393}]}


<h5> Request nft collection overview

In [12]:
try:
    response_overview = requests.get(url_nft_overview, headers=headers).json()
    #response.raise_for_status()
    print(response_overview)
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
    

{'collection': 'boredape-baseclub', 'name': 'Bored Ape Base Club', 'description': 'Bored Ape Base Club | 🎁[$BABC](https://x.com/oxboredapebc) | [$BBKC](https://x.com/oxboredapebc)🎁 | Bored Base Kennel Club', 'image_url': 'https://i.seadn.io/s/raw/files/afb4939e07f432a7803974d2b8b3c72c.gif?w=500&auto=format', 'banner_image_url': 'https://i.seadn.io/s/primary-drops/0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c/31864168:about:media:992eb49c-6dd6-4d14-abe1-654d8ce71574.png?w=500&auto=format', 'owner': '0xd954bb185991e0e11b90704f35beec3ce54078d2', 'safelist_status': 'verified', 'category': 'pfps', 'is_disabled': False, 'is_nsfw': False, 'trait_offers_enabled': True, 'collection_offers_enabled': True, 'opensea_url': 'https://opensea.io/collection/boredape-baseclub', 'project_url': '', 'wiki_url': '', 'discord_url': '', 'telegram_url': '', 'twitter_username': 'OxBoredApeBC', 'instagram_username': 'boredapebaseclub', 'contracts': [{'address': '0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c', 'chain':

<p> Creating dataframe with collection stats

In [7]:
df_total_status = pd.json_normalize(response_stats['total'])
#$2,637.70 USD (ETH price 2025-02-09)
eth_price = 2637.7
df_total_status['volume - USD'] = (df_total_status['volume'] * eth_price).round(2)
df_total_status ['floor_price - USD'] = (df_total_status['floor_price'] * eth_price).round(2)
df_total_status['collection'] = slug_collection
display(df_total_status)

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol,volume - USD,floor_price - USD,collection
0,60.534992,16534,0.003661,593,45.51355,0.00595,ETH,159673.15,15.69,boredape-baseclub


<p> Creating dataframe with collection overview

In [14]:
df_overview = pd.json_normalize(response_overview)
display(df_overview)

,collection,name,description,image_url,banner_image_url,owner,safelist_status,category,is_disabled,is_nsfw,...,editors,fees,required_zone,total_supply,created_date,rarity.strategy_id,rarity.strategy_version,rarity.calculated_at,rarity.max_rank,rarity.tokens_scored
0,boredape-baseclub,Bored Ape Base Club,Bored Ape Base Club | 🎁[$BABC](https://x.com/o...,https://i.seadn.io/s/raw/files/afb4939e07f432a...,https://i.seadn.io/s/primary-drops/0x0be0bb109...,0xd954bb185991e0e11b90704f35beec3ce54078d2,verified,pfps,False,False,...,[0xd954bb185991e0e11b90704f35beec3ce54078d2],"[{'fee': 2.5, 'recipient': '0x0000a26b00c1f0df...",0x000056f7000000ece9003ca63978907a00ffd100,9992,2024-04-15,openrarity,0.7.5,2025-01-01T12:07:13.444973,9963,9963


In [ ]:
df_overview_status = df_total_status.merge(df_overview, on="collection",how="left")


,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol,volume - USD,floor_price - USD,collection,...,editors,fees,required_zone,total_supply,created_date,rarity.strategy_id,rarity.strategy_version,rarity.calculated_at,rarity.max_rank,rarity.tokens_scored
0,60.534992,16534,0.003661,593,45.51355,0.00595,ETH,159673.15,15.69,boredape-baseclub,...,[0xd954bb185991e0e11b90704f35beec3ce54078d2],"[{'fee': 2.5, 'recipient': '0x0000a26b00c1f0df...",0x000056f7000000ece9003ca63978907a00ffd100,9992,2024-04-15,openrarity,0.7.5,2025-01-01T12:07:13.444973,9963,9963


In [ ]:
def sanitize_filename(name):
    return re.sub(r'[\\/:*?"<>|]', '', name)

safe_slug = sanitize_filename(slug_collection)

datasource_dir = "datasource"

if not os.path.exists(datasource_dir):
    os.makedirs(datasource_dir)

dataframe_json = df_overview_status.to_json(safe_slug +"_overview_status.json")